STS


# Zadanie
## Celem zadania jest pozyskanie odpowiedzi potrzebnych do uzupełnienia plików:
#### ➢ group_scores_name_surname – to zadanie na rozgrzewkę
#### ➢ game_scores_name_surname – to zadanie sprawdzające elementarne umiejętności potrzebne w naszej pracy
#### ➢ interactions_name_surname – to zadanie wymagające odrobinę więcej kreatywności :)

# Group_scores
#### W tej części prosimy Cię o zwrócenie kilku szczegółów dla każdej z kategorii, którą znajdziesz w pliku game_groups.csv. Kolejno chcemy mieć informację o:
####  - count_of_games - liczbie gier w kategorii
####  - count_of_movies – liczbie filmów w kategorii
####  - count_of_comments – liczbie komentarzy dla danej kategorii gier esportowych w przesłanej bazie
####  - count_of_distinct_users - każdy komentarz został opublikowany przez pewnego użytkownika, który kryje się za wartością podaną w kolumnie user. Chcemy poznać liczbę różnych użytkowników, którzy choć raz skomentowali film w danej kategorii gier esportowych.
####  - average_no_of_distinct_users_per_game – chcemy wiedzieć ilu średnio (różnych) użytkowników wypowiada się pod wybranymi filmami dotyczącymi gry w każdej kategorii gier esportowych.

In [2]:
# Imports
import pandas as pd
from pandasql import sqldf

# SQL query function
def pysqldf(q):
    return sqldf(q, globals())
#    pysqldf = lambda q: sqldf(q, globals())

In [3]:
# folders, dataframes
data_path = 'data/' # data folder
data_frames  = ["comments","game_groups","game_movies","games"] #data csc files
output_folder = 'output/'# output folder

In [4]:
# read tables
comments = pd.read_csv(data_path + "comments.csv", encoding = 'utf-8',sep = ',')
game_groups = pd.read_csv(data_path + "game_groups.csv", encoding = 'utf-8',sep = ',')
game_movies = pd.read_csv(data_path + "game_movies.csv",encoding = 'utf-8',sep = ',')
games = pd.read_csv(data_path + "games.csv",encoding = 'utf-8',sep = ',')

In [5]:
# read "zadania"
group_scores_Tomasz_Cwik = pd.read_csv("group_scores_name_surname.csv",encoding = 'utf-8',sep = ',')
game_scores_Tomasz_Cwik = pd.read_csv("game_scores_name_surname.csv",encoding = 'utf-8',sep = ',')
interactions_Tomasz_Cwik = pd.read_csv("interactions_name_surname.csv",encoding = 'utf-8',sep = ',')

#group_scores_Tomasz_Cwik 


# #1 Group_scores
W tej części prosimy Cię o zwrócenie kilku szczegółów dla każdej z kategorii, którą znajdziesz w pliku game_groups.csv. Kolejno chcemy mieć informację o:
#### count_of_games - liczbie gier w kategorii
! Użyłem left join aby upewnić się, że wezmę wszystkie kategorie z game_groups

In [6]:
group_scores_Tomasz_Cwik = game_groups 
group_scores_Tomasz_Cwik

,id_game_group,game_group_name
0,6,Sports games
1,5,Real-time strategy
2,1,Fighting games
3,3,Multiplayer online battle arena
4,8,Others
5,2,First-person shooters
6,7,Third-person shooters
7,4,Racing


In [9]:
# ! Użyłem left join aby upewnić się, że wezmę wszystkie kategorie z game_groups

q = """

    select gg.game_group_name, gg.id_game_group, count(*) count_of_games
    from game_groups as gg
     
    left join games as g
    on g.id_game_group = gg.id_game_group
    group by gg.id_game_group

"""


count_of_games = pysqldf(q) 
count_of_games

,game_group_name,id_game_group,count_of_games
0,Fighting games,1,5
1,First-person shooters,2,14
2,Multiplayer online battle arena,3,7
3,Racing,4,6
4,Real-time strategy,5,3
5,Sports games,6,5
6,Third-person shooters,7,3
7,Others,8,8


#### count_of_movies – liczbie filmów w kategorii
! left join aby pobrać wszystkie kategorie

In [10]:
q = """

   select gg.game_group_name, gg.id_game_group, count(*) as count_of_movies
   from
        game_groups as gg
            left join
        games as g on gg.id_game_group = g.id_game_group
            left join
        game_movies as gm on g.id_game = gm.id_game
    
    group by gg.id_game_group

"""
count_of_movies = pysqldf(q)
count_of_movies

,game_group_name,id_game_group,count_of_movies
0,Fighting games,1,25
1,First-person shooters,2,70
2,Multiplayer online battle arena,3,35
3,Racing,4,30
4,Real-time strategy,5,15
5,Sports games,6,25
6,Third-person shooters,7,15
7,Others,8,40


#### count_of_comments – liczbie komentarzy dla danej kategorii gier esportowych w przesłanej bazie

In [11]:
q = """select gg.game_group_name,gg.id_game_group, count(id_comment) as count_of_comments

        
        from
            game_groups as gg
        left join
            games as g on gg.id_game_group = g.id_game_group
        left join
            game_movies as gm on g.id_game = gm.id_game
        left join
            comments as c  on gm.id_movie = c.id_movie
            
        group by gg.id_game_group

"""


count_of_comments = pysqldf(q)
count_of_comments

,game_group_name,id_game_group,count_of_comments
0,Fighting games,1,47760
1,First-person shooters,2,47352
2,Multiplayer online battle arena,3,15516
3,Racing,4,3558
4,Real-time strategy,5,12788
5,Sports games,6,17798
6,Third-person shooters,7,10314
7,Others,8,14678


#### count_of_distinct_users
każdy komentarz został opublikowany przez pewnego użytkownika, który kryje się za wartością podaną w kolumnie user. Chcemy poznać liczbę różnych użytkowników, którzy choć raz skomentowali film w danej kategorii gier esportowych.

In [12]:
q = """select gg.game_group_name,gg.id_game_group, count(DISTINCT USER) as count_of_distinct_users
        from
            game_groups as gg
        left join
            games as g on gg.id_game_group = g.id_game_group
        left join
            game_movies as gm on g.id_game = gm.id_game
        left join
            comments as c  on gm.id_movie = c.id_movie
            
        group by gg.id_game_group

"""


count_of_distinct_users = pysqldf(q)
count_of_distinct_users

,game_group_name,id_game_group,count_of_distinct_users
0,Fighting games,1,19330
1,First-person shooters,2,22505
2,Multiplayer online battle arena,3,7360
3,Racing,4,1686
4,Real-time strategy,5,5044
5,Sports games,6,8400
6,Third-person shooters,7,5007
7,Others,8,7095


### average_no_of_distinct_users_per_game 
chcemy wiedzieć ilu średnio (różnych) użytkowników wypowiada się pod wybranymi filmami dotyczącymi gry w każdej kategorii gier esportowych.
Tomek Ćwik - liczę średnią na poziomie kategorii dla wszystkich unikalnych użytkowników,

In [13]:
 q = """

    with distinct_user_per_movie as (

    select gg.id_game_group, gm.id_movie, count(DISTINCT USER) as count_of_distinct_users

        
        from
            game_groups as gg
        left join
            games as g on gg.id_game_group = g.id_game_group
        left join
            game_movies as gm on g.id_game = gm.id_game
        left join
            comments as c  on gm.id_movie = c.id_movie
            
        group by gg.id_game_group, gm.id_movie
    )
    
    select id_game_group, avg(count_of_distinct_users) as average_no_of_distinct_users_per_game
        from distinct_user_per_movie
    group by id_game_group
   
        
"""

average_no_of_distinct_users_per_game = pysqldf(q)
average_no_of_distinct_users_per_game

,id_game_group,average_no_of_distinct_users_per_game
0,1,909.200000
1,2,324.485714
2,3,212.114286
3,4,56.500000
4,5,409.000000
5,6,337.440000
6,7,334.133333
7,8,177.925000


In [11]:
# merging data
group_scores_Tomasz_Cwik = group_scores_Tomasz_Cwik.merge(count_of_games.
                               merge(count_of_movies.
                                     merge(count_of_comments.
                                           merge(count_of_distinct_users.
                                                 merge(average_no_of_distinct_users_per_game , on = "id_game_group"), on = "id_game_group"), on = "id_game_group"), on = "id_game_group"), on = "id_game_group")




In [12]:
# Save to csv
column_list_to_save = ["game_group_name", "count_of_games", "count_of_movies", "count_of_comments", "count_of_distinct_users" , "average_no_of_distinct_users_per_game"]
group_scores_Tomasz_Cwik.to_csv(path_or_buf  = "group_scores_Tomasz_Cwik.csv", columns = column_list_to_save, index = False, encoding = 'utf-8' )

